In [2]:
import pandas as pd

In [2]:
!pip install easyocr
!pip install joblib
!pip install pillow
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 17.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 30.5 MB/s eta 0:00:00


In [8]:
import easyocr
import joblib
import os
from PIL import Image
import numpy as np
import easyocr
import numpy as np
import re
import os
from PIL import Image
from io import BytesIO




PRICE_PATTERN = r"(.+?)\s+(\d+[.,]\d{2})\s*€?$"


def extract_text_from_image(image_bytes: bytes) -> list[str]:
    """
    Convertit une image en liste de lignes de texte.
    """
    # Initialisation du lecteur EasyOCR
    reader = easyocr.Reader(['fr'], gpu=False, model_storage_directory= r"C:\Users\lenovo\Desktop\file_rouge_new\projects\app\services\easyocr_models")
    image = Image.open(BytesIO(image_bytes)).convert("RGB")
    image_np = np.array(image)

    lines = reader.readtext(image_np, detail=0)
    return lines


# Fonction de separtion des elements de l'image en format article : prix 
def extract_items(lines: list[str]) -> list[dict]:
    items = []

    for line in lines:
        match = re.search(PRICE_PATTERN, line)
        if match:
            label = match.group(1).strip()
            amount = float(match.group(2).replace(",", "."))

            items.append({
                "label": label,
                "amount": amount
            })

    return items

import re

BLACKLIST = [
    "total", "tva", "cb", "carte", "merci", "date", "ticket"
]

def clean_receipt_lines(lines: list[str]) -> list[str]:
    cleaned = []

    for line in lines:
        line = line.strip()

        if not re.search(r"\d", line):
            continue

        if any(word in line.lower() for word in BLACKLIST):
            continue

        cleaned.append(line)

    return cleaned



In [ ]:
!pip install google.colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.4 MB/s eta 0:00:0000:010:01


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
# Load the model and vectorizer
model = joblib.load(r"C:\Users\lenovo\Desktop\file_rouge_new\ml_project\models\expense_categorizer_model.pkl")
vectorizer = joblib.load(r"C:\Users\lenovo\Desktop\file_rouge_new\ml_project\models\vectorizer.pkl")

# Initialize OCR reader
reader = easyocr.Reader(['fr', 'en'])  # Support French and English


In [ ]:
def extract_receipt_from_image_bytes(image_bytes: bytes):
    raw_text = extract_text_from_image(image_bytes)
    cleaned_text = clean_receipt_lines(raw_text)
    items = extract_items(cleaned_text)

    return {
        "items": items,
        "count": len(items)
    }
